In [61]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.layers import Input, Dense, Reshape, Dropout, CuDNNLSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils

In [62]:
def get_notes(path, output):
    notes = []

    for file in glob.glob("MIDIs/Touhou MIDIs/*.mid"):
        midi = converter.parse(file)
    
        print("Parsing %s" % file)

        notes_to_parse = None

#         try: # file has instrument parts
#             s2 = instrument.partitionByInstrument(midi)
#             notes_to_parse = s2.parts[0].recurse() 
#         except: # file has notes in a flat structure
#             notes_to_parse = midi.flat.notes

        notes_to_parse = midi.flat.notes
        
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            if isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open(output, 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [54]:
chords = get_notes("MIDIs/Touhou MIDIs/*.mid",'data/touhou')

Parsing MIDIs/Touhou MIDIs/th06_01.mid
Parsing MIDIs/Touhou MIDIs/th06_02.mid
Parsing MIDIs/Touhou MIDIs/th06_03.mid
Parsing MIDIs/Touhou MIDIs/th06_04.mid
Parsing MIDIs/Touhou MIDIs/th06_05.mid
Parsing MIDIs/Touhou MIDIs/th06_06.mid
Parsing MIDIs/Touhou MIDIs/th06_07.mid
Parsing MIDIs/Touhou MIDIs/th06_08.mid
Parsing MIDIs/Touhou MIDIs/th06_09.mid
Parsing MIDIs/Touhou MIDIs/th06_10.mid
Parsing MIDIs/Touhou MIDIs/th06_11.mid
Parsing MIDIs/Touhou MIDIs/th06_12.mid
Parsing MIDIs/Touhou MIDIs/th06_13.mid
Parsing MIDIs/Touhou MIDIs/th06_14.mid
Parsing MIDIs/Touhou MIDIs/th06_15.mid
Parsing MIDIs/Touhou MIDIs/th06_16.mid
Parsing MIDIs/Touhou MIDIs/th06_17.mid
Parsing MIDIs/Touhou MIDIs/th07_01.mid
Parsing MIDIs/Touhou MIDIs/th07_02.mid
Parsing MIDIs/Touhou MIDIs/th07_03.mid
Parsing MIDIs/Touhou MIDIs/th07_04.mid
Parsing MIDIs/Touhou MIDIs/th07_05.mid
Parsing MIDIs/Touhou MIDIs/th07_06.mid
Parsing MIDIs/Touhou MIDIs/th07_07.mid
Parsing MIDIs/Touhou MIDIs/th07_08.mid
Parsing MIDIs/Touhou MIDI

In [63]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab)/2) / (float(n_vocab)/2)
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def generate_notes(model, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)
    
    # Get pitch names and store in a dictionary
    pitchnames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = numpy.append(pattern,index)
        #pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [66]:
class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_discriminator(self):

        model = Sequential()
        model.add(CuDNNLSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(CuDNNLSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)

        return Model(noise, seq)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load and convert the data
        notes = chords
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Training the model
        for epoch in range(epochs):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)

            # Print the progress and save into loss lists
            if epoch % sample_interval == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
              self.disc_loss.append(d_loss[0])
              self.gen_loss.append(g_loss)
        
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        # Get pitch names and store in a dictionary
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        
        # Use random noise to generate sequences
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes = [int_to_note[int(x)] for x in pred_notes]
        
        create_midi(pred_notes, 'gan_final')
        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

In [67]:
gan = GAN(rows=100)    
gan.train(epochs=5000, batch_size=32, sample_interval=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_3 (CuDNNLSTM)     (None, 100, 512)          1054720   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              4202496   
_________________________________________________________________
dense_8 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total para

InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' used by node cu_dnnlstm_1/CudnnRNN (defined at /usr/local/lib/python3.6/dist-packages/keras/layers/cudnn_recurrent.py:517) with these attrs: [T=DT_FLOAT, input_mode="linear_input", direction="unidirectional", rnn_mode="lstm", is_training=true, seed2=0, seed=87654321, dropout=0]
Registered devices: [CPU, XLA_CPU]
Registered kernels:
  <no registered kernels>

	 [[cu_dnnlstm_1/CudnnRNN]]

Errors may have originated from an input operation.
Input Source operations connected to node cu_dnnlstm_1/CudnnRNN:
 cu_dnnlstm_1/concat_1 (defined at /usr/local/lib/python3.6/dist-packages/keras/layers/cudnn_recurrent.py:60)	
 cu_dnnlstm_1/ExpandDims_1 (defined at /usr/local/lib/python3.6/dist-packages/keras/layers/cudnn_recurrent.py:487)	
 cu_dnnlstm_1/transpose (defined at /usr/local/lib/python3.6/dist-packages/keras/layers/cudnn_recurrent.py:484)	
 cu_dnnlstm_1/ExpandDims_2 (defined at /usr/local/lib/python3.6/dist-packages/k

In [ ]:
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.25

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))
create_midi(chords, "chords")

In [5]:
import midi
import numpy as np
class MidiCoordinator(object):
    def __init__(self, lowerBound, upperBound):
        self._lowerBound = lowerBound
        self._upperBound = upperBound
        self._span = upperBound-lowerBound
        
    def midiToMatrix(self, midifile, squash=True):
        schema = midi.read_midifile(midifile)
        totalTime = [track[0].tick for track in schema]
        posns = [0 for track in schema]
        matrix = []
        time = 0
        
        state = [[0,0] for x in range(self._span)]
        matrix.append(state)
        end = False
        
        while not end:
            if time % (schema.resolution/4) == (schema.resolution / 8):
                oldstate = state
                state = [[oldstate[x][0],0] for x in range(self._span)]
                matrix.append(state)
            for i in range(len(totalTime)): #For each track
                if end:
                    break
                while totalTime[i] == 0:
                    track = schema[i]
                    pos = posns[i]
    
                    evt = track[pos]
                    if isinstance(evt, midi.NoteEvent):
                        if (evt.pitch < self._lowerBound) or (evt.pitch >= self._upperBound):
                            pass
                        else:
                            if isinstance(evt, midi.NoteOffEvent) or evt.velocity == 0:
                                state[evt.pitch-self._lowerBound] = [0, 0]
                            else:
                                state[evt.pitch-self._lowerBound] = [1, 1]
                    elif isinstance(evt, midi.TimeSignatureEvent):
                        if evt.numerator not in (2, 4):
                            end = True
                            break
                    try:
                        totalTime[i] = track[pos + 1].tick
                        posns[i] += 1
                    except IndexError:
                        totalTime[i] = None
    
                if totalTime[i] is not None:
                    totalTime[i] -= 1
    
            if all(t is None for t in totalTime):
                break
    
            time += 1
    
        S = np.array(matrix)
        statematrix = np.hstack((S[:, :, 0], S[:, :, 1]))
        statematrix = np.asarray(statematrix).tolist()
        return matrix

    def matrixToMidi(self, matrix, name="example"):
        matrix = np.array(matrix)
        if not len(matrix.shape) == 3:
            matrix = np.dstack((matrix[:, :self._span], matrix[:, self._span:]))
        matrix = np.asarray(matrix)
        pattern = midi.Pattern()
        track = midi.Track()
        pattern.append(track)
        tickscale = 55
        
        lastcmdtime = 0
        prevstate = [[0,0] for x in range(self._span)]
        for time, state in enumerate(matrix + [prevstate[:]]):  
            offNotes = []
            onNotes = []
            for i in range(self._span):
                n = state[i]
                p = prevstate[i]
                if p[0] == 1:
                    if n[0] == 0:
                        offNotes.append(i)
                    elif n[1] == 1:
                        offNotes.append(i)
                        onNotes.append(i)
                elif n[0] == 1:
                    onNotes.append(i)
            for note in offNotes:
                track.append(midi.NoteOffEvent(tick=(time-lastcmdtime)*tickscale, pitch=note + self._lowerBound))
                lastcmdtime = time
            for note in onNotes:
                track.append(midi.NoteOnEvent(tick=(time-lastcmdtime)*tickscale, velocity=40, pitch=note + self._lowerBound))
                lastcmdtime = time
                
            prevstate = state
        
        eot = midi.EndOfTrackEvent(tick=1)
        track.append(eot)
                                                                
        midi.write_midifile("{}.mid".format(name), pattern) 

In [12]:
from tqdm import tqdm
import glob
import numpy as np

class SongImporter(object):
    def __init__(self, path, midi_coordinator):
        self._path = path
        self._midi_coordinator = midi_coordinator
        
    def getSongs(self):
        files = glob.glob('MIDIs/Touhou MIDIs/*.mid')
        songs = []
        for file in tqdm(files):
            try:
                song = np.array(self._midi_coordinator.midiToMatrix(file))
                if np.array(song).shape[0] > 50:
                    songs.append(song)
            except Exception as e:
                raise e           
        return songs

In [46]:
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
from tqdm import tqdm
import numpy as np

class MusicGenerator(object):
    def __init__(self, midi_coordinator, learning_rate = 0.005, num_timesteps = 15, batch_size = 100, num_of_epochs = 200):
        self.learning_rate = tf.constant(0.005, tf.float32)
        self.num_timesteps = num_timesteps
        self.batch_size = batch_size
        self.num_of_epochs = num_of_epochs
        self._midi_coordinator = midi_coordinator

        # Define TF variables and placeholders
        self._visible_dim = 2*(midi_coordinator._upperBound - midi_coordinator._lowerBound)*num_timesteps
        self._hidden_dim = 50
        self._input  = tf.placeholder(tf.float32, [None, self._visible_dim], name="input")
        self._weights  = tf.Variable(tf.random_normal([self._visible_dim, self._hidden_dim], 0.01), name="weights")
        self._hidden_bias = tf.Variable(tf.zeros([1, self._hidden_dim],  tf.float32, name="hidden_bias"))
        self._visible_bias = tf.Variable(tf.zeros([1, self._visible_dim],  tf.float32, name="visible_bias"))
        
        visible_cdstates = self.gibsSampling(1) 
        hidden_states = self.callculate_state(tf.sigmoid(tf.matmul(self._input, self._weights) + self._hidden_bias)) 
        hidden_cdstates = self.callculate_state(tf.sigmoid(tf.matmul(visible_cdstates, self._weights) + self._hidden_bias)) 
        
        size = tf.cast(tf.shape(self._input)[0], tf.float32)
        weights_delta  = tf.multiply(self.learning_rate/size, tf.subtract(tf.matmul(tf.transpose(self._input), hidden_states), tf.matmul(tf.transpose(visible_cdstates), hidden_cdstates)))
        visible_bias_delta = tf.multiply(self.learning_rate/size, tf.reduce_sum(tf.subtract(self._input, visible_cdstates), 0, True))
        hidden_bias_delta = tf.multiply(self.learning_rate/size, tf.reduce_sum(tf.subtract(hidden_states, hidden_cdstates), 0, True))
        
        self._updates = [self._weights.assign_add(weights_delta), self._visible_bias.assign_add(visible_bias_delta), self._hidden_bias.assign_add(hidden_bias_delta)]
    
    def gibsSampling(self, number_of_iterations):
        counter = tf.constant(0)
        [_, _, visible_cdstates] = control_flow_ops.while_loop(lambda count, num_iter, *args: count < num_iter,
                                             self.singleStep, [counter, tf.constant(number_of_iterations), self._input])
        
        # Stop tensorflow from propagating gradients back through the gibbs step
        visible_cdstates = tf.stop_gradient(visible_cdstates) 
        return visible_cdstates
    
    def callculate_state(self, probability):
        return tf.floor(probability + tf.random_uniform(tf.shape(probability), 0, 1))
    
    def singleStep(self, count, index, input_indexed):
        hidden_states = self.callculate_state(tf.sigmoid(tf.matmul(input_indexed, self._weights) + self._hidden_bias))
        visible_cdstates = self.callculate_state(tf.sigmoid(tf.matmul(hidden_states, tf.transpose(self._weights)) + self._visible_bias))
        return count+1, index, visible_cdstates
    
    def generateSongs(self, songs):
        with tf.Session() as sess:
            init = tf.global_variables_initializer()
            sess.run(init)
            for epoch in tqdm(range(self.num_of_epochs)):
                for song in songs:
                    song = np.array(song)
                    song = song[:int(np.floor(song.shape[0]/self.num_timesteps)*self.num_timesteps)]
                    print(song.shape)
                    song = np.reshape(song, [song.shape[0]//self.num_timesteps, song.shape[1]*self.num_timesteps])
                    for i in range(1, len(song), self.batch_size): 
                        tr_x = song[i:i+self.batch_size]
                        sess.run(self._updates, feed_dict={self._input: tr_x})
        
            sample = self.gibsSampling(1).eval(session=sess, feed_dict={self._input: np.zeros((50, self._visible_dim))})
            for i in range(sample.shape[0]):
                if not any(sample[i,:]):
                    continue
                matrix = np.reshape(sample[i,:], (self.num_timesteps, 2*self._midi_coordinator._span))
            self._midi_coordinator.matrixToMidi(matrix, "song_{}".format(i)) 

In [16]:
coordinator = MidiCoordinator(24, 102)
importer = SongImporter('midi_songs', coordinator)


imported_songs = importer.getSongs()

100%|██████████| 42/42 [00:15<00:00,  1.93it/s]


In [47]:
generator = MusicGenerator(coordinator)
generator.generateSongs(imported_songs)

  0%|          | 0/200 [00:00<?, ?it/s]

(540, 78, 2)


ValueError: cannot reshape array of size 84240 into shape (36,1170)